In [1]:
import os
import sys
import numpy as np
from tqdm import tqdm
from PIL import Image
import random

from utils import predictor

val_dir = "/home/workthu/zy/dataset/imagenet/imagenet_valset/"
Pred = predictor.predictor(device_name = "cuda:0", model_name = "vit_tiny_patch16_224")

Model loaded on cuda:0


/home/hxy/anaconda3/envs/timm/lib/python3.7/site-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [2]:
random.seed(1)
%mkdir results-ViT-t

def ablation(n_abl, layers):
    rand1_correct = 0
    rand2_correct = 0
    rand3_correct = 0
    topn_correct = 0

    zero_heads = np.arange(0, 3).tolist()

    for i in tqdm(range(1000)):
        label = Pred.cat_labels[i]
        class_dir = val_dir + label
        imgs_path = sorted(os.listdir(class_dir))

        attns = np.load("./attns/ViT-t/" + label + ".npy")

        for j in range(50):
            attn = attns[11, j]
            top10 = np.argsort(-attn[1:])[:n_abl] + 1

            img_path = class_dir + "/" + imgs_path[j]

            prob, cat, _, _ = Pred.predict_one(img_path, zero_lines = [layers, zero_heads, random.sample(range(1, 197), n_abl)])
            rand1_correct += (cat == i)
            prob, cat, _, _ = Pred.predict_one(img_path, zero_lines = [layers, zero_heads, random.sample(range(1, 197), n_abl)])
            rand2_correct += (cat == i)
            prob, cat, _, _ = Pred.predict_one(img_path, zero_lines = [layers, zero_heads, random.sample(range(1, 197), n_abl)])
            rand3_correct += (cat == i)
            prob, cat, _, _ = Pred.predict_one(img_path, zero_lines = [layers, zero_heads, top10])
            topn_correct += (cat == i)
    
        if (i + 1) % 100 == 0:
            print(i + 1, rand1_correct, rand2_correct, rand3_correct, topn_correct)
            
            f = open("results-ViT-t/abl-n" + str(n_abl) + "-l" + str(layers) + ".txt", "a")
            f.write(str(i + 1) + " " + str(rand1_correct) + " " + str(rand2_correct) + " " + str(rand3_correct) + " " +
                    str(topn_correct) + "\n")
            f.close()
    
    return(rand1_correct, rand2_correct, rand3_correct, topn_correct)

In [3]:
ablation(n_abl = 10, layers = [11])

 10%|███████▋                                                                     | 100/1000 [09:51<1:26:02,  5.74s/it]

100 4081 4079 4092 4049


 20%|███████████████▍                                                             | 200/1000 [19:34<1:15:51,  5.69s/it]

200 8083 8081 8098 8007


 30%|███████████████████████                                                      | 300/1000 [29:23<1:18:22,  6.72s/it]

...

 90%|█████████████████████████████████████████████████████████████████████▎       | 900/1000 [1:28:47<09:26,  5.66s/it]

900 33456 33453 33502 33233


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:38:48<00:00,  5.93s/it]

1000 37210 37203 37260 36977


(37210, 37203, 37260, 36977)

In [4]:
ablation(n_abl = 20, layers = [11])

 10%|███████▋                                                                     | 100/1000 [09:55<1:26:32,  5.77s/it]

100 4084 4087 4080 4030


 20%|███████████████▍                                                             | 200/1000 [20:00<1:15:37,  5.67s/it]

200 8089 8088 8088 7982


 30%|███████████████████████                                                      | 300/1000 [29:46<1:18:31,  6.73s/it]

...

 90%|█████████████████████████████████████████████████████████████████████▎       | 900/1000 [1:28:08<09:29,  5.69s/it]

900 33449 33472 33453 33074


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:37:51<00:00,  5.87s/it]

1000 37205 37228 37206 36797


(37205, 37228, 37206, 36797)

In [ ]:
ablation(n_abl = 39, layers = [11])

 10%|███████▋                                                                     | 100/1000 [09:49<1:25:37,  5.71s/it]

100 4082 4081 4075 4029


 20%|███████████████▍                                                             | 200/1000 [19:29<1:15:25,  5.66s/it]

200 8091 8084 8064 7964


 30%|███████████████████████                                                      | 300/1000 [29:12<1:17:43,  6.66s/it]

...

 70%|█████████████████████████████████████████████████████▉                       | 700/1000 [1:09:37<28:49,  5.77s/it]

700 26485 26507 26489 26136


 80%|█████████████████████████████████████████████████████████████▌               | 800/1000 [1:19:16<18:47,  5.64s/it]

800 30003 30010 30011 29578


 85%|█████████████████████████████████████████████████████████████████▍           | 850/1000 [1:24:06<14:20,  5.73s/it]

In [ ]:
ablation(n_abl = 59, layers = [11])

In [ ]:
ablation(n_abl = 10, layers = [10, 11])

In [ ]:
ablation(n_abl = 20, layers = [10, 11])